In [2]:
import cv2
import os
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
from glob import glob
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, BatchNormalization, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


In [3]:
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
import math

# Define a more aggressive data augmentation for the training data
train_datagenerator = ImageDataGenerator(
    rescale=1.0/255,
    shear_range=0.2,
    zoom_range=0.5,
    horizontal_flip=True,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=[0.5, 1.5],
    fill_mode='nearest'
)

test_datagenerator = ImageDataGenerator(rescale = 1.0/255)

In [15]:
train_data = train_datagenerator.flow_from_directory('/kaggle/input/bone-fracture-detection-using-xrays/archive (6)/train',
                                                    target_size=(224, 224),
                                                    batch_size = 50,
                                                    class_mode = 'categorical')
val_data = test_datagenerator.flow_from_directory('/kaggle/input/bone-fracture-detection-using-xrays/archive (6)/val',
                                                  target_size=(224, 224),
                                                    batch_size = 50,
                                                    class_mode = 'categorical')

Found 8863 images belonging to 2 classes.
Found 600 images belonging to 2 classes.


In [33]:
pretrained_model = tf.keras.applications.DenseNet121(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)

29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [34]:
def lr_schedule(epoch):
    initial_lr = 0.001
    decay_factor = 0.9
    return initial_lr * math.pow(decay_factor, epoch)


In [35]:
for layer in pretrained_model.layers[:-100]: #fine tune last 100 layers
    layer.trainable = True


In [40]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [37]:
model = tf.keras.Sequential([
    pretrained_model,
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])


In [38]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [41]:
history = model.fit(
    train_data,
    epochs=10,
    validation_data=val_data,
    callbacks=[early_stopping, LearningRateScheduler(lr_schedule)]
)

Epoch 1/10


W0000 00:00:1711491003.613428     103 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


 13/178 ━━━━━━━━━━━━━━━━━━━━ 23:48 9s/step - accuracy: 0.5735 - loss: 0.9346  

W0000 00:00:1711491107.771989     101 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


178/178 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5410 - loss: 0.7997

W0000 00:00:1711491229.349897     100 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


178/178 ━━━━━━━━━━━━━━━━━━━━ 415s 1s/step - accuracy: 0.5411 - loss: 0.7993 - val_accuracy: 0.5717 - val_loss: 0.6468 - learning_rate: 0.0010
Epoch 2/10
178/178 ━━━━━━━━━━━━━━━━━━━━ 121s 652ms/step - accuracy: 0.6150 - loss: 0.6422 - val_accuracy: 0.6550 - val_loss: 0.5611 - learning_rate: 9.0000e-04
Epoch 3/10
178/178 ━━━━━━━━━━━━━━━━━━━━ 121s 651ms/step - accuracy: 0.6804 - loss: 0.5839 - val_accuracy: 0.6233 - val_loss: 0.6160 - learning_rate: 8.1000e-04
Epoch 4/10
178/178 ━━━━━━━━━━━━━━━━━━━━ 120s 648ms/step - accuracy: 0.7408 - loss: 0.5144 - val_accuracy: 0.6150 - val_loss: 1.4656 - learning_rate: 7.2900e-04
Epoch 5/10
178/178 ━━━━━━━━━━━━━━━━━━━━ 119s 642ms/step - accuracy: 0.8044 - loss: 0.4194 - val_accuracy: 0.6217 - val_loss: 1.2055 - learning_rate: 6.5610e-04
Epoch 6/10
178/178 ━━━━━━━━━━━━━━━━━━━━ 121s 651ms/step - accuracy: 0.8561 - loss: 0.3262 - val_accuracy: 0.6700 - val_loss: 0.8894 - learning_rate: 5.9049e-04
Epoch 7/10
178/178 ━━━━━━━━━━━━━━━━━━━━ 122s 655ms/step - 

In [46]:
from PIL import Image
import numpy as np
image_path = '/kaggle/input/bone-fracture-detection-using-xrays/archive (6)/val/fractured/1-rotated1-rotated3-rotated3.jpg'
image = Image.open(image_path)
image = image.resize((224, 224))  # Resize to match the input shape of the model
image = np.array(image) / 255.0  # Normalize pixel values to [0, 1]
image = np.expand_dims(image, axis=0)  # Add batch dimension

# Perform prediction
predictions = model.predict(image)

# Interpret predictions
predicted_class = np.argmax(predictions)  # Get the index of the class with highest probability
class_names = ['fractured', 'normal']  # Define class names
predicted_class_name = class_names[predicted_class]

print("Predicted class:", predicted_class_name)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Predicted class: fractured
